In [ ]:
import os, sys

# if os.path.exists('analysis'):
#     os.system('rm -rf analysis')

# !git clone https://github.com/element-fi/analysis.git

# sys.path.insert(1, os.getcwd())

In [ ]:
import json, numbers, math, time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sim import YieldSimulator, ElementPricingModel, Market, YieldSpacev2PricingModel, YieldSpacev2MinFeePricingModel

np.random.seed(1)

In [ ]:
config = {
    'step_size': 0.001,
    'min_fee': 0.,
    'max_fee': 0.5,
    't_min': 0.001,
    't_max': 1.0,
    'base_asset_price': 2500., # aka market price
    'min_target_liquidity': 100000.,
    'max_target_liquidity': 10000000.,
    'min_target_volume': 2e5,
    'max_target_volume': 6e5,
    'min_apy': 5, # default 0.5
    'max_apy': 5, # default 50
    'min_vault_age': 1., # default 0.
    'max_vault_age': 1, # default 2.
    'min_vault_apy': 0.,
    'max_vault_apy': 10.,
    'min_pool_age': 0.,
    'max_pool_age': 0.5,
    'precision': 8,
    'pricing_model_name': 'YieldSpace',
    'tokens': ['base', 'fyt'],
    'trade_direction': 'out',
    'days_until_maturity': 90,
    'num_trading_days': 90, # should be <= days_until_maturity
    'random_seed': 3
}

simulator = YieldSimulator(**config)

In [ ]:
start_time = time.time()
num_runs = 0

simulator = YieldSimulator(**config)
simulator.set_sim_params()
for pricing_model in ['Element', 'YieldSpace', 'YieldSpaceMinFee']:
    # for apy in range(1, 11, 1):
    for vault_apy in [5]:
        override_dict = {
            'vault_apy': vault_apy,
            'pricing_model_name': pricing_model,
        }
        simulator.run_simulation(override_dict)
        num_runs += 1

end_time = time.time()
print(f'Total time = {end_time-start_time} seconds; and {(end_time-start_time)/num_runs} seconds per run')

In [ ]:
trades = pd.DataFrame.from_dict(simulator.analysis_dict)
trades.head(5).T

In [ ]:
display(trades.tail(1).T)

In [ ]:
trades['time_diff'] = trades.time_until_end.diff()
trades['time_diff_shift'] = trades.time_until_end.shift(-1).diff()
trades['fee_in_bps'] = trades.fee / trades.out_without_fee * 100 * 100
trades.loc[(trades.time_diff>0) | (trades.time_diff_shift>0) | (trades.index == trades.index.max()),:]

In [ ]:
general_columns = ['run_number','model_name','num_orders','t_stretch','target_liquidity','target_daily_volume','start_apy','current_apy','fee_percent','init_vault_age','vault_apy','pool_age','days_until_maturity','num_trading_days'] 
# create runs dataframe from the last trade in each run (using time_diff instead would pick the 1st)
runs = trades.loc[(trades.time_diff_shift>0),:].copy().loc[:,general_columns]
# variables that change per trade represent the last value in the run, rename the useful ones
runs.rename(columns={'current_apy':'ending_apy','num_orders':'total_orders'}, inplace=True)
# general_columns=runs.columns.to_list()
display(runs.head(1).T)

In [ ]:
do_not_aggregate = ['run_number','model_name','num_orders']
dfg = trades.groupby(general_columns[0])[[x for x in general_columns if x not in do_not_aggregate]].std()
display(dfg.head(1).T)

In [ ]:
trade_columns = list(set(trades.columns) - set(general_columns) - set(do_not_aggregate))
display(trade_columns)

In [ ]:
# do mean things
mean_columns = ['time_until_end','normalizing_constant','conversion_rate','spot_price','out_without_fee_slippage','x_reserves','out_without_fee','fee','simulation_time'\
    ,'trade_amount','out_with_fee','day','fee_in_bps','y_reserves','total_supply']
dfg_mean = trades.groupby(['run_number'])[mean_columns].mean()
dfg_mean.columns = ['mean_'+col for col in dfg_mean.columns]

# do sum things
sum_columns = ['fee','out_with_fee','out_without_fee','out_without_fee_slippage','trade_amount']
dfg_sum = trades.groupby(['run_number'])[sum_columns].mean()
dfg_sum.columns = ['sum_'+col for col in dfg_sum.columns]

dfg = pd.concat([dfg, dfg_mean, dfg_sum], axis=1)
dfgt = dfg.T
display(dfgt)
dfgt.to_csv('dfgt.csv')